In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from wordcloud import WordCloud,STOPWORDS
from subprocess import check_output
import re

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Sentiment.csv to Sentiment (1).csv


In [ ]:
#nltk.download()

In [4]:
import io
data = pd.read_csv(io.BytesIO(uploaded['Sentiment.csv']))

In [5]:
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [6]:
data.shape

(13871, 21)

In [6]:
data = data[['text', 'sentiment']]

In [7]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [8]:
def cleantxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '',text)# removed @mentions
    text = re.sub(r'#', '',text)# removed symbol
    text = re.sub(r'RT[\s]+', '',text)# removed RT
    text = re.sub(r'https?:\/\/\s+', '',text)# removed the hyperlink
    text = re.sub(r':+', '',text)# removed : symbol
    text = re.sub(r'--+', '',text)# removed : symbol
    text = re.sub(r'http', '',text)
    return text
data["text"] = data["text"].apply(cleantxt)

In [9]:
data.head()

,text,sentiment
0,How did everyone feel about the Climate Chang...,Neutral
1,Didn't catch the full GOPdebate last night. H...,Positive
2,No mention of Tamir Rice and the GOPDebate wa...,Neutral
3,That Carly Fiorina is trending hours after H...,Positive
4,GOPDebate w/ delivered the highest ratings i...,Positive


In [10]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

data["subjectivity"] = data["text"].apply(getSubjectivity)
data["polarity"] = data["text"].apply(getPolarity)

In [11]:
data.head()

,text,sentiment,subjectivity,polarity
0,How did everyone feel about the Climate Chang...,Neutral,0.158333,0.125
1,Didn't catch the full GOPdebate last night. H...,Positive,0.305556,0.450
2,No mention of Tamir Rice and the GOPDebate wa...,Neutral,1.000000,0.100
3,That Carly Fiorina is trending hours after H...,Positive,0.100000,0.000
4,GOPDebate w/ delivered the highest ratings i...,Positive,0.000000,0.000


In [12]:
train,test = train_test_split(data,test_size = 0.1)
train = train[train.sentiment != "Neutral"]

train_pos = train[train["sentiment"]=="positive"]
train_pos = train_pos["text"]

train_neg = train[train["sentiment"]=="negative"]
train_neg = train_neg["text"]

In [15]:
text = []
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    text.append((words_without_stopwords, row.sentiment))

test_pos = test[ test['sentiment'] == 'Positive']
test_pos = test_pos['text']
test_neg = test[ test['sentiment'] == 'Negative']
test_neg = test_neg['text']

In [16]:
def get_words_in_text(text):
    all = []
    for (words, sentiment) in text:
        all.extend(words)
    return all

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_word_features(get_words_in_text(text))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [17]:
training_set = nltk.classify.apply_features(extract_features,text)

In [18]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [21]:
neg_cnt = 0
pos_cnt = 0
for obj in test_neg: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'Negative'): 
        neg_cnt = neg_cnt + 1
for obj in test_pos: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'Positive'): 
        pos_cnt = pos_cnt + 1
        
print('[Negative]: %s/%s '  % (neg_cnt, len(test_neg)))
print('[Positive]: %s/%s '  % (pos_cnt, len(test_pos)))

[Negative]: 807/835 
[Positive]: 71/225 
